<a href="https://colab.research.google.com/github/seongyeon1/twitterNLP/blob/main/preprocessing_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DataFrame
import pandas as pd

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Tensorflow
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [ ]:
# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_IN_PATH = '/content/drive/MyDrive/ColabNotebooks/datasets/sentiments/'

In [ ]:
train_data = pd.read_csv(DATA_IN_PATH + 'train.csv')
test_data = pd.read_csv(DATA_IN_PATH + 'test.csv')
train_data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [ ]:
df = pd.concat([train_data, test_data])

In [ ]:
!pip install unidecode
!pip install ktrain
!pip install wordninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 59.2 MB/s 
     |████████████████████████████████| 22.3 MB 68.5 MB/s 
     |████████████████████████████████| 981 kB 59.9 MB/s 
     |████████████████████████████████| 263 kB 68.4 MB/s 
     |████████████████████████████████| 2.8 MB 47.6 MB/s 
     |████████████████████████████████| 1.2 MB 62.1 MB/s 
     |████████████████████████████████| 468 kB 66.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 3.3 MB 50.3 MB/s 
     |████████████████████████████████| 880 kB 68.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.2-py3-none-any.whl size=25313063 sha256=99ba12e104383a3e13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 541 kB 5.2 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=af3832be9077e47b27876cf9098b4d1e87e589b66fb25467614d4bed4c7a3f5d
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [ ]:
import re
import unidecode
import ktrain
from ktrain import text
import wordninja
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

### 전처리

In [ ]:
df_train = df[:7920]
df_test = df[7920:]

In [ ]:
# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

### 불용어 처리

In [ ]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

stop_words = set(stopwords.words("english"))
stop_words_list = ['no', 'nor', 'not', 'don', "don't", 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
                   "hadn't", 'hasn', "hasn't", "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
                   "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

for i in stop_words_list:
  stop_words.remove((i))

print(stop_words)

{'such', 'if', 'those', 'through', 'its', 'our', 'does', 'this', 'him', 'did', 'now', 'we', 'above', 'just', 'after', 'of', 'against', 'on', 'myself', 'having', 'hers', 'some', 'ain', 'once', 'between', 'own', 'ourselves', 'who', 'all', 've', 'am', 'because', 'their', 'to', 'a', 'hadn', 'from', 'yourselves', 'when', 'ours', 'other', 'these', 'they', 'can', 'in', 'o', 'is', 'again', 'she', 'have', 'it', 'that', 'then', 'why', "that'll", 'more', 'how', 'were', 'or', 'both', 'doing', 'll', 'my', 'as', 'off', 'for', 'too', 's', 'below', 'was', 'itself', 'by', 'with', "it's", 'be', 'but', 'do', 'himself', 'under', 'what', 'had', 'which', 'most', 'are', 'and', 't', "she's", 'y', 'into', 'me', 'theirs', 'whom', 'down', 'd', 'where', 'he', 'his', 'yours', 'here', 'i', 'so', 'your', 'during', 'while', 'each', 'yourself', "you'd", 'further', 'at', 'until', 'ma', 'the', 'before', 'few', 'being', 'you', 'themselves', "you've", 'an', 'only', 'there', 'them', 'very', 'should', "should've", 'will', '

In [ ]:
def clean_tweet(text, stem=False):

    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    text = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|\^\^)', 'good', text)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    text = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', 'laugh', text)
    # Sad -- :-(, : (, :(, ):, )-:
    text = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', 'bad', text)
    # Cry -- :,(, :'(, :"(
    text = re.sub(r'(:,\(|:\'\(|:"\()', 'sad', text)

    
    # lower-case all characters
    text=text.lower()
    
    # remove twitter handles
    text= re.sub(r'@\S+', '',text) 
    
    # remove urls
    text= re.sub(r'http\S+', '',text) 
    text= re.sub(r'pic.\S+', '',text)
      
    # replace unidecode characters
    text=unidecode.unidecode(text)
      
    # regex only keeps characters
    text= re.sub(r"[^A-Za-z0-9]+", ' ',text)
    
    # keep words with length>1 only
    text=re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ') 

    # split words like 'whatisthis' to 'what is this'
    def preprocess_wordninja(sentence):      
        def split_words(x):
            x=wordninja.split(x)
            x= [word for word in x if len(word)>1]
            return x
        new_sentence=[ ' '.join(split_words(word)) for word in sentence.split() ]
        return ' '.join(new_sentence)
    
    text=preprocess_wordninja(text)
 
    # regex removes repeated spaces, strip removes leading and trailing spaces
    text= re.sub("\s[\s]+", " ",text).strip()  
    tokens = []
    for token in text.split():
        if token not in stop_words:
          if stem:
              tokens.append(stemmer.stem(token))
          else:
              tokens.append(token)
    return " ".join(tokens)

In [ ]:
%%time
df.tweet = df.tweet.apply(lambda x: clean_tweet(x))

CPU times: user 4.02 s, sys: 14.6 ms, total: 4.04 s
Wall time: 4.03 s


In [ ]:
df

,id,label,tweet
0,1,0.0,fingerprint pregnancy test android apps beauti...
1,2,0.0,finally trans paran silicon case good thanks u...
2,3,0.0,love would go talk make memories unplug relax ...
3,4,0.0,i'm wired know i'm george made way iphone cute...
4,5,1.0,amazing service apple won't even talk question...
...,...,...,...
1948,9869,NaN,samsung galaxy note explodes burns year old th...
1949,9870,NaN,available hood ie check iphone case music disc...
1950,9871,NaN,goes crack right across screen could actually ...
1951,9872,NaN,said adobe big time may well include apple
